In [9]:
!pip install plotly
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json

## This is a notebook used just to visualise the solution and create the chart in the output folder

In [10]:
# graph with demand

# read output
df = pd.read_csv("shift.csv")

# Convert to datetime format
df['start_date_time'] = pd.to_datetime(df['start_date_time'])
df['end_date_time'] = pd.to_datetime(df['end_date_time'])
df['staff_id'] = df['staff_id'].astype(str)

# Calculate the duration of each shift in hours
df['shift_duration'] = (df['end_date_time'] - df['start_date_time']).dt.total_seconds() / 3600

fig = px.timeline(df, 
                  x_start='start_date_time', 
                  x_end='end_date_time', 
                  y='staff_id', 
                  title='Staff Work Shifts',
                  labels={'staff_id': 'Staff', 'start_date_time': 'Shift Start', 'end_date_time': 'Shift End'},
                  color='staff_id',
                  text='shift_duration')

fig = go.Figure(fig)

# read demand
demand_df = pd.read_csv("../data/demand.csv")
demand_df['date_time'] = pd.to_datetime(demand_df['date_time'])

fig.add_trace(go.Scatter(
    x=demand_df['date_time'],
    y=demand_df['demand'],
    mode="lines",
    name="Demand",
    line=dict(color="gray", width=1),
    yaxis="y2"  # Secondary y-axis
))

# add metrics in title
with open("metrics.json") as f:
    metrics = json.load(f)


# Update layout to add and configure the secondary y-axis
fig.update_layout(
    yaxis=dict(
        title="Staff",
        anchor="x",
        domain=[0.0, 1.0]  # Ensure it spans the full height
    ),
    yaxis2=dict(
        title="Demand",
        overlaying="y",  # Share the same x-axis
        side="right"     # Display the secondary y-axis on the right
    ),
    xaxis_title="Time",
    title=f"Staff Schedule with Demand on Y-Axis WDC {metrics['WDC']}, WOR {metrics['WOR']}",
    showlegend=False
)
#fig.update_layout(showlegend=True, xaxis_title='Time', yaxis_title='Staff ID', xaxis_type='date')

fig.show()